In [ ]:
import cv2
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import datetime
from tensorflow import keras
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Model , Sequential
from tensorflow.keras.layers import Dense , Flatten , Lambda , Input
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from glob import glob

In [ ]:
IMAGE_SIZE = [200,200]

train_path = 'chest x-ray/train'
test_path = 'chest x-ray/test'

vgg19_model = VGG19(
  weights = 'imagenet',
  input_shape=IMAGE_SIZE + [3],
  include_top=False
)
#freezing all the layers when training
for layer in vgg19_model.layers:
  layer.trainable = False

folders = glob('chest x-ray/train/*')

x = Flatten()(vgg19_model.output)

prediction = Dense(len(folders),activation = 'softmax')(x)

model = Model(inputs = vgg19_model.input , outputs = prediction)

model.summary()

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
#don't forgert to raise the output : text line limit to be able to run the test fully (30 --> 20 000)
from keras.preprocessing.image import ImageDataGenerator

#
train_datagen = ImageDataGenerator(rescale = 1./200,shear_range = 0.2,zoom_range = 0.3 , horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./200)

#attributing the right directory to train/test
training_set = train_datagen.flow_from_directory('chest x-ray/train',target_size = (200,200),batch_size = 32 , class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('chest x-ray/test',target_size = (200,200),batch_size = 32 , class_mode = 'categorical')

#training the whole thing with our vgg19, doing 5 iterations
model.fit(training_set,validation_data= test_set,epochs = 5,steps_per_epoch = len(training_set),validation_steps =len(test_set))

In [ ]:
#Saving your model
model.save('chest_images_vgg19_v2.h5')